In [1]:
# https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models
# https://cookbook.openai.com/examples/using_tool_required_for_customer_service
from spire.doc import *
from spire.doc.common import *
from dotenv import dotenv_values
import openai
from openai import OpenAI
import numpy as np
import os
import sqlite3
import ast 
import pandas as pd
import json
import tiktoken
# Create a Document instance
document = Document()

C:\Users\hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
api_key = dotenv_values("../api_data.env")
openai.api_key = api_key['OPEN_AI_KEY']

In [3]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", openai.api_key))

In [4]:
# models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-3.5-turbo"

# Function calling 

In [5]:
import sqlite3
conn = sqlite3.connect("alerting_db")
print("Opened database successfully")

Opened database successfully


In [6]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)
print(database_schema_string)

Table: alerts_table
Columns: alert_id, alert_date, trader_id, deal_number, model_name
Table: deals_table
Columns: alert_id, deal_date, deal_number, price, volume


In [7]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

# Embedding

### Reading the Embedding df with text and Embedding

In [8]:
df = pd.read_csv('Embedding_df/embedding_Macquarie_Group_announces_$A3.csv').drop(columns = ['Unnamed: 0'])
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [9]:
df.head()

,text,embedding
0,"FY24 net profit of $A3,522 million, down 32% o...","[[0.026646699756383896, 0.009173554368317127, ..."
1,Bank Level 2 CET1 ratio 13.6% (Harmonised5: 18...,"[[0.010210090316832066, 0.041706670075654984, ..."
2,"Annuity-style activities, which are undertaken...","[[0.0324883796274662, 0.04309939965605736, 0.0..."
3,Assets under management at 31 March 2024 were ...,"[[0.05020987242460251, 0.02893919125199318, 0...."
4,Macquarie’s financial position exceeds the Aus...,"[[0.0031701396219432354, 0.024023061618208885,..."


### Function to fetch the closed text paras for the given input

In [10]:
# models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-3.5-turbo"
# search function
from scipy import spatial 
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below articles on the SOW of a Company to answer questions"'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the macquarie annual report news."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    response_message = response.choices[0].message.content
    return response_message

# Tools

In [19]:
# The tools our customer service LLM will use to communicate
tools = [
{
  "type": "function",
  "function": {
    "name": "Answer_generic_questions",
    "description": """Use this to speak to the user to give them information using the ongoing conversation and a apt response.
                      Read the whole previous conversation till the lastest user query and see if you answer.
                      User might ask to summarize any document, do spell check or write a mailer for him/her""",
    "parameters": {
      "type": "object",
      "properties": {
        "message": {
          "type": "string",
          "description": "Text of message to send to user. Can cover multiple topics."
        }
      },
      "required": ["message"]
    }
  }
},
{
  "type": "function",
  "function": {
    "name": "get_information",
    "description": "Used to get instructions to deal with the user's problem.",
    "parameters": {
      "type": "object",
      "properties": {
        "information": {
          "type": "string",
          "description": """The user wants to know information about the Macquarie annual report.
                            Use the embedding search functionality to answer properly. """
        }
      },
      "required": [
        "information"
      ]
    }
  }
},
{
    "type": "function",
    "function": {
        "name": "ask_database",
        "description": "Use this function to answer user questions about Alerts and Deals in trading data. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
}
]

# Example instructions that the customer service assistant can consult for relevant customer problems
# INSTRUCTIONS = [ {"type": "get_query",
#                   "instructions": """• Ask the user to describe the scenerio for which they need the sql query .
# • Offer the user the sql query.
# • Thank the user for contacting support and invite them to reach out with any future queries."""},
#                 {"type": "get_information",
#                  "instructions": """• Greet the user and ask how you can assist them today.
# • Listen carefully to the user's query and clarify if necessary.
# • Provide accurate and clear information based on the user's questions.
# • Offer to assist with any additional questions or provide further details if needed.
# • Ensure the user is satisfied with the information provided.
# • Thank the user for contacting support and invite them to reach out with any future queries.""" },
#                {"type": "ask_database",
#                  "instructions": """• Greet the user and ask how you can assist them today.
# • Listen carefully to the user's query and clarify if necessary.
# • Provide accurate and clear SQL Query based on the user's questions.
# • Offer to assist with any additional questions or provide further details if needed.
# • Ensure the user is satisfied with the information provided.
# • Thank the user for contacting support and invite them to reach out with any future queries.""" }]

### Functions to decide with tool to pick and to generate response according to the tool

In [12]:
# def submit_user_message(user_query,conversation_messages=[]):
#     """Message handling function which loops through tool calls until it reaches one that requires a response.
#     Once it receives respond=True it returns the conversation_messages to the user."""

#     # Initiate a respond object. This will be set to True by our functions when a response is required
#     respond = False
    
#     user_message = {"role":"user","content": user_query}
#     conversation_messages.append(user_message)

#     print(f"User: {user_query}")

#     while respond is False:

#         # Build a transient messages object to add the conversation messages to
#         messages = [
#             {
#                 "role": "system",
#                 "content": assistant_system_prompt
#             }
#         ]

#         # Add the conversation messages to our messages call to the API
#         [messages.append(x) for x in conversation_messages]

#         # Make the ChatCompletion call with tool_choice='required' so we can guarantee tools will be used
#         response = client.chat.completions.create(model=GPT_MODEL
#                                                   ,messages=messages
#                                                   ,temperature=0
#                                                   ,tools=tools
#                                                   ,tool_choice='required'
#                                                  )

        
#         messages.append(response.choices[0].message)
#         print("conversation_messages in submit_user_message: ",messages)
      
#         respond, messages = execute_function(response.choices[0].message,messages)
    
#     return messages1

def execute_function(function_calls,messages):
    """Wrapper function to execute the tool calls"""

    for function_call in function_calls.tool_calls:
    
        function_id = function_call.id
        function_name = function_call.function.name
        function_arguments = json.loads(function_call.function.arguments)
    
        if function_name == 'get_information':

            respond = True
            
            
            # instruction_name will have the user query:
            instruction_name = function_arguments['information']
            
            
            response_message_embedding = ask(instruction_name)
            messages.append({
                        "role":"assistant", 
                        "content": response_message_embedding
                        })
            
            print(f"Assistant: {response_message_embedding}")
            print("\n\n")
            
        elif function_name == 'ask_database':

            respond = True
    
            instruction_name = function_arguments['query']
            messages.append({
                        "role":"assistant", 
                        "content": instruction_name
                        })
             
            print(f"Assistant: {instruction_name}")
            print("\n\n")
            
        elif function_name == 'Answer_generic_questions':

            respond = True
            instruction_name = function_arguments['message']
            messages.append({
                    "role":"assistant", 
                    "content": instruction_name
                    })

    
            print(f"Assistant: {instruction_name}")
            print("\n\n")
    
    return (respond, messages)
    

### Defining the System Message 

In [13]:
assistant_system_prompt = """You are a user service assistant. Your role is to answer user questions politely and competently.
You should follow these instructions to solve the case:
- Understand their problem and get the relevant instructions.
- Follow the instructions to solve the user's problem.
- Help them with any other problems or close the case.

Only call a tool once in a single message.
If you need to fetch a piece of information from a system or document that you don't have access to, give a clear, confident answer with some dummy values."""

### Create a simple GUI to interact

In [21]:

import PySimpleGUI as sg

sg.theme('GreenTan') # give our window a spiffy set of colors

layout = [[sg.Text('Your output will go here', size=(40, 1))],
          [sg.Output(size=(110, 20), font=('Helvetica 10'))],
          [sg.Multiline(size=(70, 5), enter_submits=True, key='-QUERY-', do_not_clear=False),
           sg.Button('SEND', button_color=(sg.YELLOWS[0], sg.BLUES[0]), bind_return_key=True),
           sg.Button('EXIT', button_color=(sg.YELLOWS[0], sg.GREENS[0]))]]

window = sg.Window('Chat window', layout, font=('Helvetica', ' 13'), default_button_element_size=(8,2), use_default_focus=False)

#conversation_messages = []
messages = [
    {
                "role": "system",
                "content": assistant_system_prompt
    }
]
try:
    while True:     # The Event Loop
        event, values = window.read()
        if event in (sg.WIN_CLOSED, 'EXIT'):            # quit if exit button or X
            break
            #window.close()
        if event == 'SEND':
            #print("inside send")
            # Initiate a respond object. This will be set to True by our functions when a response is required
            respond = False
            
            user_question = values['-QUERY-'].rstrip()
            user_message = {"role":"user","content": user_question}
            messages.append(user_message)

            print(f"User: {user_question}")
            #print("\n\n")

            
            # Make the ChatCompletion call with tool_choice='required' so we can guarantee tools will be used
            response = client.chat.completions.create(model=GPT_MODEL
                                                      ,messages=messages
                                                      ,temperature=0
                                                      ,tools=tools
                                                      ,tool_choice='required'
                                                     )
            
            print("Response: ",response)
            #print("conversation_messages in submit_user_message: ",messages)
            respond, messages = execute_function(response.choices[0].message,messages)
            print("\n\n")
            
except Exception as e:
    print("Exception occured :",e)
    window.close()
    
window.close()

In [15]:
messages

[{'role': 'system',
  'content': "You are a user service assistant. Your role is to answer user questions politely and competently.\nYou should follow these instructions to solve the case:\n- Understand their problem and get the relevant instructions.\n- Follow the instructions to solve the user's problem.\n- Help them with any other problems or close the case.\n\nOnly call a tool once in a single message.\nIf you need to fetch a piece of information from a system or document that you don't have access to, give a clear, confident answer with some dummy values."}]

In [16]:
## Tool call for information:

# tool_calls=[ChatCompletionMessageToolCall(id='call_9DckYbESC1pkmsQGTaXQOWvF', function=Function(arguments='{"information":"Difference between CGM and MAM contribution in profit by number"}', name='get_information'), type='function')])]

In [17]:
## tool call for ask databse:

# tool_calls=[ChatCompletionMessageToolCall(id='call_BrcSvu8KvMirkkGJGThKRpOV', function=Function(arguments='{"query":"SELECT model_name, COUNT(alert_id) AS alert_count FROM alerts_table GROUP BY model_name ORDER BY alert_count DESC LIMIT 1;"}', name='ask_database'), type='function')